# projet 
### Skander Meziou


# Partie 1 - Extraction des donées 

les donées des paroles etant uniquement disponibles sur l'api j'ai du les extraire directement depuis l'api 

In [1]:
import requests
import json
import time
from datetime import datetime
import sqlite3

api_url = "http://wasabi.i3s.unice.fr/api/v1/song_all/"
params = {
    "project": "lyrics,genre,publicationDateAlbum"
}

sqlite_file = "db.sqlite"
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

start = c.execute("SELECT COUNT(*) FROM songs").fetchone()[0]

counter = 0
try:
    while True:
        # Fetch data from the API
        response = requests.get(f"{api_url}{start}", params=params)
        if response.status_code == 200:
            print(datetime.now(), f"Fetched data from {start}")
            new_data = response.json()
            if not new_data:
                # Exit loop if no new data is returned
                break
            c.executemany("INSERT INTO songs (json) VALUES (?)", [(json.dumps(song),) for song in new_data])
            counter += 1
            if counter % 100 == 0:
                conn.commit()
            start += len(new_data)
        else:
            print(datetime.now(), f"Failed to fetch data: {response.status_code}. Retrying in 60 seconds...")
            time.sleep(60)
except KeyboardInterrupt:
    print("Exiting...")
finally:
    conn.commit()
    conn.close()
    print("Done")

KeyboardInterrupt: 

In [6]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('db.sqlite')
cursor = conn.cursor()

# Afficher un exemple de ligne de la table 'songs'
cursor.execute("SELECT json FROM songs LIMIT 1")
ligne = cursor.fetchone()

# Affichage de l'exemple de contenu JSON
print("Exemple de contenu de la colonne 'json' :")
print(ligne[0])

# Fermer la connexion
conn.close()


Exemple de contenu de la colonne 'json' :
{"_id": "5714dec325ac0d8aee3804e7", "lyrics": "Turn it up<br>I don&apos;t know where you&apos;re plugging in<br>Listen up<br>Yeah, we&apos;re all set to begin<br>Turn it up<br>Because we hope you like it loud<br>Tune it up<br>Any second now<br>Making like<br>Cheeky Monkey grins<br>And for tonight<br>I&apos;d love to let you in", "publicationDateAlbum": "1997"}


In [7]:
import sqlite3
import json
from collections import Counter
import re
import pandas as pd

# Connexion à la base de données
conn = sqlite3.connect('db.sqlite')
cursor = conn.cursor()

# Récupérer toutes les lignes de la colonne 'json'
cursor.execute("SELECT json FROM songs")
lignes = cursor.fetchall()

# Initialiser un DataFrame pour stocker les informations extraites
data = []

for ligne in lignes:
    # Charger chaque ligne JSON
    record = json.loads(ligne[0])
    
    # Vérifier si les champs nécessaires existent
    if 'lyrics' in record and 'publicationDateAlbum' in record:
        paroles = record['lyrics']
        annee = record['publicationDateAlbum']
        
        # Ajouter les données au DataFrame
        data.append({'annee': annee, 'paroles': paroles})

# Convertir en DataFrame
df = pd.DataFrame(data)

# Afficher un aperçu du DataFrame
print(df.head())

# Fermer la connexion
conn.close()



  annee                                            paroles
0  1997  Turn it up<br>I don&apos;t know where you&apos...
1  1997  Sick of you, how old do you think you are?<br>...
2  1997  My name is Jason<br>I&apos;ll rock yer face in...
3  1997  Got to get out more<br>Get in the place<br>Got...
4  1997  Bad idea (x4) <br><br>Ninety-nine per cent is ...


# Partie 2 - transformation en json exploitable 

afin de pouvoir utiliser les donées pour les lire en json et avoir le nombre de fois que chaque parole apparait je les transforme en json 

In [18]:
import sqlite3
import json
from collections import Counter
import re

# Connexion à la base de données
conn = sqlite3.connect('db.sqlite')
cursor = conn.cursor()

# Récupérer le nombre total de lignes dans la table
cursor.execute("SELECT COUNT(*) FROM songs")
total_lignes = cursor.fetchone()[0]

# Initialiser un dictionnaire pour stocker les mots par genre et année
mots_par_genre_annee = {}

# Blacklist de mots trop courants à ignorer
blacklist = {
    'br', 'i', 'the', 'a', 'an', 'and', 'to', 'in', 'of', 'for', 'on', 'with', 'at', 'by', 'is', 
    'it', 'this', 'that', 'are', 'was', 'as', 'be', 'have', 'has', 'or', 'quot', 's', 't', 'm', 
    'll', 'd', 're', 've', 'will', 'can', 'just', 'so', 'my', 'me', 'if', 'no', 'out', 'from', 
    'like', 'well', 'oh', 'all', 'not', 'don', 'do', 'now', 'yeah', 'up', 'one', 'get', 'go', 
    'la', 'da', 'ohh', 'uh', 'eh', 'gonna', 'ain', 'cause', 'ooh', 'ya', 'got','apos','you'
}

batch_size = 10000
offset = 0
lignes_traitees = 0

while True:
    cursor.execute(f"SELECT json FROM songs LIMIT {batch_size} OFFSET {offset}")
    lignes = cursor.fetchall()
    
    if not lignes:
        break

    for ligne in lignes:
        try:
            record = json.loads(ligne[0])
            # Vérifie que les champs existent et sont valides
            if 'lyrics' in record and 'publicationDateAlbum' in record and 'genre' in record:
                if not isinstance(record['lyrics'], str) or not isinstance(record['publicationDateAlbum'], str):
                    continue
                genre = record['genre']
                if isinstance(genre, list):
                    genre = genre[0] if len(genre) > 0 else None
                if not genre:
                    continue
                
                annee = record['publicationDateAlbum']
                paroles = record['lyrics']
                
                mots = re.findall(r'\w+', paroles.lower())
                mots_frequences = Counter(word for word in mots if word not in blacklist)
                
                if genre not in mots_par_genre_annee:
                    mots_par_genre_annee[genre] = {}
                if annee not in mots_par_genre_annee[genre]:
                    mots_par_genre_annee[genre][annee] = Counter()
                
                mots_par_genre_annee[genre][annee].update(mots_frequences)
        except json.JSONDecodeError:
            print("Erreur de décodeur JSON, enregistrement ignoré.")
        except Exception as e:
            print(f"Erreur inattendue : {e}")

    lignes_traitees += len(lignes)
    pourcentage = (lignes_traitees / total_lignes) * 100
    print(f"Progression : {pourcentage:.2f}% ({lignes_traitees}/{total_lignes} lignes traitées)")
    
    offset += batch_size

conn.close()

with open('data_mots.json', 'w') as file:
    json.dump(mots_par_genre_annee, file)

print("Traitement terminé.")


Progression : 0.48% (10000/2099287 lignes traitées)
Progression : 0.95% (20000/2099287 lignes traitées)
Progression : 1.43% (30000/2099287 lignes traitées)
Progression : 1.91% (40000/2099287 lignes traitées)
Progression : 2.38% (50000/2099287 lignes traitées)
Progression : 2.86% (60000/2099287 lignes traitées)
Progression : 3.33% (70000/2099287 lignes traitées)
Progression : 3.81% (80000/2099287 lignes traitées)
Progression : 4.29% (90000/2099287 lignes traitées)
Progression : 4.76% (100000/2099287 lignes traitées)
Progression : 5.24% (110000/2099287 lignes traitées)
Progression : 5.72% (120000/2099287 lignes traitées)
Progression : 6.19% (130000/2099287 lignes traitées)
Progression : 6.67% (140000/2099287 lignes traitées)
Progression : 7.15% (150000/2099287 lignes traitées)
Progression : 7.62% (160000/2099287 lignes traitées)
Progression : 8.10% (170000/2099287 lignes traitées)
Progression : 8.57% (180000/2099287 lignes traitées)
Progression : 9.05% (190000/2099287 lignes traitées)
Pr